In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import MSELoss
import pandas as pd
from AttentiveFP import get_smiles_dicts, get_smiles_array, num_atom_features, num_bond_features  # assumed to be your actual featurizer
from AttentiveFP import Fingerprint  # assumed to be your attentive FP model

In [2]:
# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset ---
class MoleculeDataset(Dataset):
    def __init__(self, smiles_list, targets, feature_dicts):
        self.smiles_list = smiles_list
        self.targets = targets
        self.feature_dicts = feature_dicts
        self.x_atom, self.x_bond, self.x_atom_index, self.x_bond_index, self.x_mask, _ = get_smiles_array(smiles_list, feature_dicts)

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.x_atom[idx], dtype=torch.float32),
            torch.tensor(self.x_bond[idx], dtype=torch.float32),
            torch.tensor(self.x_atom_index[idx], dtype=torch.long),
            torch.tensor(self.x_bond_index[idx], dtype=torch.long),
            torch.tensor(self.x_mask[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

# --- Load data ---
df = pd.read_csv("../data/LD50_Zhu.csv")  # your dataset path
smiles_list = df["smiles"].tolist()
targets = df["active"].tolist()
feature_dicts = get_smiles_dicts(smiles_list)

# --- Dataloader ---
dataset = MoleculeDataset(smiles_list, targets, feature_dicts)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# --- Model ---
model = Fingerprint(
    radius=5,
    T=3,
    input_feature_dim=num_atom_features(),
    input_bond_dim=num_bond_features(),
    fingerprint_dim=200,
    output_units_num=1,
    p_dropout=0.1
).to(device)

optimizer = Adam(model.parameters(), lr=1e-3)
loss_fn = MSELoss()

# --- Training loop ---
for epoch in range(30):
    model.train()
    total_loss = 0

    for atom, bond, atom_deg, bond_deg, mask, target in loader:
        atom = atom.to(device)
        bond = bond.to(device)
        atom_deg = atom_deg.to(device)
        bond_deg = bond_deg.to(device)
        mask = mask.to(device)
        target = target.to(device)

        _, pred, _ = model(atom, bond, atom_deg, bond_deg, mask)
        loss = loss_fn(pred.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1} - Loss: {total_loss:.4f}")


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx